In [3]:
#attempt to modify NTRU Encrypt by using R_p = F_p[S_N] rather than R_p = F_p[C_N] = F_p[x]/(x^N-1)

In [4]:
#the group of units of the symmetric group algebra are computed in Corollary 3.1, 3.2 of
#"Structure of the Unit Group of the Symmetric Group Algebra", Abhilash, Nandakumar

In [2]:
homomorphic_encryption = False

In [12]:
#define the parameters
N=3; p=5; q=67; d=2
assert N.is_prime()
assert p.is_prime()
assert gcd(N,q) == 1 
assert gcd(p,q) == 1
assert q > (6*d+1)*p

In [13]:
#define the group
#G = CyclicPermutationGroup(N)
G = SymmetricGroup(N)

In [14]:
#create the polynomial ring \Z[x]/(x^N-1)
R = G.algebra(ZZ); R

Symmetric group algebra of order 3 over Integer Ring

In [15]:
#create the quotient polynomial ring F_p[S_N]
R_p = G.algebra(GF(p)); R_p

Symmetric group algebra of order 3 over Finite Field of size 5

In [16]:
#create the symmetric group algebra F_q[S_N]
R_q = G.algebra(Zmod(q)); R_q

Symmetric group algebra of order 3 over Ring of integers modulo 67

In [17]:
#a ternary polynomial in R = \Z[x]/(x^N-1) has d1 coeffs = +1, d2 coeffs = -1
def is_ternary(a,d1,d2):
    assert a in R
    coeffs = [pair[1] for pair in list(a)]
    return list(coeffs).count(+1) == d1 and list(coeffs).count(-1) == d2

In [7]:
#create a random unit of the symmetric group algebra over a finite field
#essentially if K_q[S_N] \cong M_{n_1}(K_{q^{m_1}}) \oplus ... \oplus M_{n_r}(K_{q^{m_r}}) then
#\mathcal{U}(K_q[S_N]) \cong GL_{n_1}(K_{q^{m_1}}) \oplus ... \oplus GL_{n_r}(K_{q^{m_r}})
#and the number n_k, m_k can be read off from the Brattelli diagram.
#however, only ~1/p elements are not invertible by determinant considerations, so most elements should be invertible
def random_unit_group_algebra(p,n,max_iter=1,ternary=False,d1=0,d2=0):
    assert p.is_prime()
    for i in range(max_iter):
        if ternary:
            random_element = rand_ternary(N,d1,d2)
        else:
            random_element = R.random_element()
        try:
            R_p(random_element).inverse()
            return random_element
        except ValueError:
            continue
    print("Could not find invertible element of symmetric group algebra.")

In [8]:
#choose a random element of \mathcal{T}(d_1,d_2)
def rand_ternary(N,d1,d2):
    group_order = G.cardinality()
    assert group_order >= d1 + d2
    coeffs = (group_order-(d1+d2))*[0] + d1*[+1] + d2*[-1]
    sigma = Permutations(group_order).random_element()
    r = sum([coeffs[sigma[i]-1]*R(G[i]) for i in range(group_order)])
    assert is_ternary(r,d1,d2)
    return r

(2,3) + 66*(1,2) + (1,2,3) + 66*(1,3,2) + (1,3)


In [23]:
#define the private key used to decrypt messages
def private_key(f):
    return (f,R_q(f).inverse())

In [24]:
#define the public key using f(x) (mod p), g(x) (mod p)
def public_key(f,g,q):
    F_q = R_q(f).inverse()
    return F_q * R_q(g)

In [25]:
print(public_key(f,g,q))

66*() + 2*(2,3) + 66*(1,2) + (1,2,3) + 66*(1,3,2)


In [26]:
#define a plaintext message m(x) with coefficients satisfying -p/2 < m_i <= p/2
#i.e. m is the center lift of a polynomial in R_p
def plaintext(m,p,N):
    assert all([-p/2 < m[i] <= +p/2 for i in range(G.cardinality())])
    return sum(m[i]*R(G[i]) for i in range(G.cardinality()))

In [27]:
#compute the ciphertext e(x)
#note m, f, g are group algebra elements
def ciphertext(m,f,g,r,p,q,N,d):
    h = public_key(f,g,q)
    e = R_q(p*h*r + m)
    return e

In [28]:
#begin decryption process
def a(f,e):
    return R_q(f*e)

In [29]:
#define the center lift of a from R_q to an element of R = \Z[x]/(x^N-1)
#map coefficients from F_q to Z/qZ (the additive group) and center lift them to \Z
def center_lift(a,modulus):
    G = R.group()
    coeffs = [a[g] for g in G]
    center_lift_coeffs = [Zmod(modulus)(coeff).lift_centered() for coeff in coeffs]
    return sum(center_lift_coeffs[i]*R(G[i]) for i in range(G.cardinality()))

In [30]:
#compute b(x) = F_p(x) * a(x) (mod p) by first center lifting a(x) to R
def b(f,a,q):
    F_p = R_p(f).inverse()
    return R_p(F_p * center_lift(a,q))

In [31]:
#define the final center_lift for decryption where e = ciphertext, f = secret key, p = prime, q = modulus
def decrypt(e,f,p,q):
    return center_lift(b(f,a(f,e),q),p)

In [42]:
#EXAMPLE

In [61]:
#ex. for cyclic group from [HPS] text with (N=7, p=3, q=41, d=2)
#f_coeffs = [-1,0,1,1,-1,0,1]
#f = sum([f_coeffs[i]*R(G[i]) for i in range(G.cardinality())]); f
#g_coeffs = [0,-1,-1,0,1,0,1]
#g = sum([g_coeffs[i]*R(G[i]) for i in range(G.cardinality())]); g
#message = [1,-1,1,1,0,-1,0]; message
#r_coeffs = [-1,1,0,0,0,-1,1]
#r = sum([r_coeffs[i]*R(G[i]) for i in range(G.cardinality())]); r

In [52]:
#create an element f of R = \ZZ[S_N] which is invertible in R_p = F_p[S_N]
f = random_unit_group_algebra(p,N,max_iter=100,ternary=True,d1=d+1,d2=d)
print(f)

() + (2,3) - (1,2) + (1,3,2) - (1,3)


In [54]:
#compute F_p(x) = f(x)^{-1} (mod p)
F_p = R_p(f).inverse(); print(F_p)
assert F_p * R_p(f) == 1

4*() + (1,2) + 4*(1,2,3) + (1,3,2) + (1,3)


In [55]:
#compute F_q(x) = f(x)^{-1} (mod q)
F_q = R_q(f).inverse(); print(F_q)
assert F_q * R_q(f) == 1

15*() + 8*(2,3) + 52*(1,2) + 15*(1,2,3) + 60*(1,3,2) + 52*(1,3)


In [56]:
#create an element g of R = \ZZ[S_N] which is invertible in R_p = F_p[S_N]
g = rand_ternary(N,d,d)
print(g)

(2,3) - (1,2) + (1,2,3) - (1,3,2)


In [59]:
message = [1,-1,1,1,0,-1]; message

[1, -1, 1, 1, 0, -1]

In [62]:
#create the polynomial from the plaintext of the message
m = plaintext(message,p,N); m

() + (2,3) - (1,2) + (1,2,3) - (1,3,2)

In [64]:
#create a random ternary element
r = rand_ternary(N,d,d); r

() - (1,2,3) + (1,3,2) - (1,3)

In [ ]:
h = public_key(f,g,q); h

In [ ]:
R_q(p*h*r + m)

In [ ]:
e = ciphertext(m,f,g,r,p,q,N,d); e

In [ ]:
center_lift(a(f,e),q)

In [ ]:
F_p

In [ ]:
F_p * center_lift(a(f,e),q)

In [ ]:
b(f,a(f,e),q)

In [ ]:
decrypt(e,f,p,q)

In [ ]:
decrypt(e,f,p,q) == m